This module works for HOURLY data:

    ** 1. Merge the pumped signals with time stamps to get vertical format for voting
    ** 2. Append the output whethere the coin was pumped or not (binary 0)

In [28]:
import os, sys
import pandas as pd 
import numpy as np
import requests 
from datetime import datetime, date, time, timedelta
import pytz
import matplotlib.pyplot as plt
import matplotlib.dates as mdate
from mpl_finance import candlestick_ohlc
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.ticker as mticker
import pickle
pd.set_option('display.max_columns', 999)
%load_ext autoreload
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Import the pumped data

### Step 1. Create the timestamp for the first hours 

In [39]:
p = pd.read_pickle("../pump_data_with_social.pkl")
p = p[['channelLink','channelTitle','currency','exchange','signalTime','pumptime_edited','timestamp']]
p.head(2)

,channelLink,channelTitle,currency,exchange,signalTime,pumptime_edited,timestamp
0,https://t.me/crypto_pump_island,Crypto Pump Island,ADX,Binance,2019-07-26T17:59:59Z,2019-07-26 17:59:59+00:00,1564163999
1,https://t.me/SignalsKingdom,Signal Kingdom,ADX,Binance,2019-07-05T17:00:32,2019-07-05 17:00:32+00:00,1562346032


Ther are .... unique points in time...

In [40]:
#Number of unique time points
len(pd.unique(p['timestamp']))

422

In [41]:
#Number of unique coins points
len(pd.unique(p['currency']))

154

In [42]:
3600*24

86400

In [43]:
1562346032 - (1562346032 % (86400))

1562284800

..Strange beheavior, must write each function one by one...

### This is a bad name. It should be HOURstamp. Corrected in the FINAL_ANALYSIS FOLDER

In [44]:
def getDaystamp(dfo: pd.DataFrame, num_days: int=1, var:str='timestamp'):
    '''
    Append a column that contains the timestamp at 12:00 AM (0:00) of the
    original var variable.
    :param df: dataframe in question 
    :param num_date: number of dates to go back to
    :param var str: name of the variable that has original timestamp
    '''
    df = dfo.copy()
    df[var+'_int'] = df[var].apply(int)
    for d in range(num_days+1):
        df[var+'_h'+str(d)] = df[var+'_int'].apply(lambda t : t - (t % (3600)) - d*(3600))
    df = df.drop(columns=var+'_int')
    return df

In [45]:
#We leave it for 1 days in advance
num_hours = 48
p = getDaystamp(p,num_hours)
p.head()

,channelLink,channelTitle,currency,exchange,signalTime,pumptime_edited,timestamp,timestamp_h0,timestamp_h1,timestamp_h2,timestamp_h3,timestamp_h4,timestamp_h5,timestamp_h6,timestamp_h7,timestamp_h8,timestamp_h9,timestamp_h10,timestamp_h11,timestamp_h12,timestamp_h13,timestamp_h14,timestamp_h15,timestamp_h16,timestamp_h17,timestamp_h18,timestamp_h19,timestamp_h20,timestamp_h21,timestamp_h22,timestamp_h23,timestamp_h24,timestamp_h25,timestamp_h26,timestamp_h27,timestamp_h28,timestamp_h29,timestamp_h30,timestamp_h31,timestamp_h32,timestamp_h33,timestamp_h34,timestamp_h35,timestamp_h36,timestamp_h37,timestamp_h38,timestamp_h39,timestamp_h40,timestamp_h41,timestamp_h42,timestamp_h43,timestamp_h44,timestamp_h45,timestamp_h46,timestamp_h47,timestamp_h48
0,https://t.me/crypto_pump_island,Crypto Pump Island,ADX,Binance,2019-07-26T17:59:59Z,2019-07-26 17:59:59+00:00,1564163999,1564160400,1564156800,1564153200,1564149600,1564146000,1564142400,1564138800,1564135200,1564131600,1564128000,1564124400,1564120800,1564117200,1564113600,1564110000,1564106400,1564102800,1564099200,1564095600,1564092000,1564088400,1564084800,1564081200,1564077600,1564074000,1564070400,1564066800,1564063200,1564059600,1564056000,1564052400,1564048800,1564045200,1564041600,1564038000,1564034400,1564030800,1564027200,1564023600,1564020000,1564016400,1564012800,1564009200,1564005600,1564002000,1563998400,1563994800,1563991200,1563987600
1,https://t.me/SignalsKingdom,Signal Kingdom,ADX,Binance,2019-07-05T17:00:32,2019-07-05 17:00:32+00:00,1562346032,1562346000,1562342400,1562338800,1562335200,1562331600,1562328000,1562324400,1562320800,1562317200,1562313600,1562310000,1562306400,1562302800,1562299200,1562295600,1562292000,1562288400,1562284800,1562281200,1562277600,1562274000,1562270400,1562266800,1562263200,1562259600,1562256000,1562252400,1562248800,1562245200,1562241600,1562238000,1562234400,1562230800,1562227200,1562223600,1562220000,1562216400,1562212800,1562209200,1562205600,1562202000,1562198400,1562194800,1562191200,1562187600,1562184000,1562180400,1562176800,1562173200
2,https://t.me/MaximumPump,Maximum Pump,ADX,Binance,2019-04-25T19:15:21,2019-04-25 19:15:21+00:00,1556219721,1556218800,1556215200,1556211600,1556208000,1556204400,1556200800,1556197200,1556193600,1556190000,1556186400,1556182800,1556179200,1556175600,1556172000,1556168400,1556164800,1556161200,1556157600,1556154000,1556150400,1556146800,1556143200,1556139600,1556136000,1556132400,1556128800,1556125200,1556121600,1556118000,1556114400,1556110800,1556107200,1556103600,1556100000,1556096400,1556092800,1556089200,1556085600,1556082000,1556078400,1556074800,1556071200,1556067600,1556064000,1556060400,1556056800,1556053200,1556049600,1556046000
3,https://t.me/bitcoinpumpgroup,Bitcoin Pump Group,WAN,Binance,2019-08-30T16:41:30Z,2019-08-30 16:41:30+00:00,1567183290,1567180800,1567177200,1567173600,1567170000,1567166400,1567162800,1567159200,1567155600,1567152000,1567148400,1567144800,1567141200,1567137600,1567134000,1567130400,1567126800,1567123200,1567119600,1567116000,1567112400,1567108800,1567105200,1567101600,1567098000,1567094400,1567090800,1567087200,1567083600,1567080000,1567076400,1567072800,1567069200,1567065600,1567062000,1567058400,1567054800,1567051200,1567047600,1567044000,1567040400,1567036800,1567033200,1567029600,1567026000,1567022400,1567018800,1567015200,1567011600,1567008000
4,https://t.me/bitcoinprofitcoach,Bitcoin Profit Coach,QSP,Binance,2019-08-29T19:46:43Z,2019-08-29 19:46:43+00:00,1567108003,1567105200,1567101600,1567098000,1567094400,1567090800,1567087200,1567083600,1567080000,1567076400,1567072800,1567069200,1567065600,1567062000,1567058400,1567054800,1567051200,1567047600,1567044000,1567040400,1567036800,1567033200,1567029600,1567026000,1567022400,1567018800,1567015200,1567011600,1567008000,1567004400,1567000800,1566997200,1566993600,1566990000,1566986400,1566982800,1566979200,1566975600,1566972000,1566968400,1566964800,1566961200,1566957600,1566954000,1566950400,1566946800,

### Step 2: Loop throug the processed social data file and merge based on timestemp
... We do not need to join by the coinID. Just the timestamps are good enough. 
Since we want to retrieve the values for all availabel timestamps

## HOURLY

In [46]:
dir_prefix = "../../Social_Data/Hourly/Processed Data/Time Series/"
social_error = []
x_final = pd.DataFrame()
t_temp = pd.DataFrame()
for filename in os.listdir(dir_prefix):
    if 'pkl' in filename:
#         print(filename)
        _, coin_name, coin_id, _ = filename.split('_')
        try: 
            print("Processing file...",filename)
            l = p
            for d in range(num_hours+1):
                if 'timestamp_h'+str(d) not in l.columns: 
                    print('Info. on hour_{} not in the left table!'.format(d))
                    continue
                x = pd.read_pickle(dir_prefix+filename)
                x.columns = [x+'_h'+str(d) if x != 'time' else 'time' for x in x.columns]
                l = pd.merge(left=l,left_on=['timestamp_h'+str(d)]
                             ,right=x, right_on=["time"]
                             ,how='inner')
                l = l.drop(columns=['time'])
            #Save the file in the processed_directory
            l['coin_name'] = coin_name
            l['coin_id'] = coin_id.replace('.pkl','')
            #Append all the data on top of each other for processing; 
            x_final = x_final.append(l)
        except: 
            social_error.append(filename)
            continue

Processing file... Hourly_MET_916769_Time.pkl
Processing file... Hourly_VIBE_336538_Time.pkl
Processing file... Hourly_SPX_683963_Time.pkl
Processing file... Hourly_EQT_105263_Time.pkl
Processing file... Hourly_DLT_220204_Time.pkl
Processing file... Hourly_PTOY_129035_Time.pkl
Processing file... Hourly_PIVX_42433_Time.pkl
Processing file... Hourly_GAM_6102_Time.pkl
Processing file... Hourly_NULS_377792_Time.pkl
Processing file... Hourly_POLY_744820_Time.pkl
Processing file... Hourly_STORJ_187347_Time.pkl
Processing file... Hourly_QSP_397757_Time.pkl
Processing file... Hourly_CHP_819737_Time.pkl
Processing file... Hourly_WTC_299397_Time.pkl
Processing file... Hourly_MTN_313967_Time.pkl
Processing file... Hourly_HC_926044_Time.pkl
Processing file... Hourly_NKT_5334_Time.pkl
Processing file... Hourly_BNT_22327_Time.pkl
Processing file... Hourly_LINK_309621_Time.pkl
Processing file... Hourly_QLC_637681_Time.pkl
Processing file... Hourly_ENJ_283116_Time.pkl
Processing file... Hourly_INK_674

Processing file... Hourly_GTO_517477_Time.pkl
Processing file... Hourly_PRO_185695_Time.pkl
Processing file... Hourly_KRL_877048_Time.pkl
Processing file... Hourly_PPT_179896_Time.pkl
Processing file... Hourly_ICN_25921_Time.pkl
Processing file... Hourly_TRX_310829_Time.pkl
Processing file... Hourly_STEEM_20333_Time.pkl
Processing file... Hourly_ELC_4439_Time.pkl
Processing file... Hourly_NPXS_731516_Time.pkl
Processing file... Hourly_CAG_341422_Time.pkl
Processing file... Hourly_ERT_328784_Time.pkl


In [47]:
print(social_error)
print(l.shape)

[]
(496, 1871)


#We might have to check whether there is a non-zero data somewhere....

In [48]:
x_final = x_final.reset_index(drop=True)
x_final = x_final.sort_values('timestamp')
x_final = x_final.drop_duplicates()

In [49]:
x_final.filter(regex="twitter_fav")

,twitter_favourites_h0,twitter_favourites_h1,twitter_favourites_h2,twitter_favourites_h3,twitter_favourites_h4,twitter_favourites_h5,twitter_favourites_h6,twitter_favourites_h7,twitter_favourites_h8,twitter_favourites_h9,twitter_favourites_h10,twitter_favourites_h11,twitter_favourites_h12,twitter_favourites_h13,twitter_favourites_h14,twitter_favourites_h15,twitter_favourites_h16,twitter_favourites_h17,twitter_favourites_h18,twitter_favourites_h19,twitter_favourites_h20,twitter_favourites_h21,twitter_favourites_h22,twitter_favourites_h23,twitter_favourites_h24,twitter_favourites_h25,twitter_favourites_h26,twitter_favourites_h27,twitter_favourites_h28,twitter_favourites_h29,twitter_favourites_h30,twitter_favourites_h31,twitter_favourites_h32,twitter_favourites_h33,twitter_favourites_h34,twitter_favourites_h35,twitter_favourites_h36,twitter_favourites_h37,twitter_favourites_h38,twitter_favourites_h39,twitter_favourites_h40,twitter_favourites_h41,twitter_favourites_h42,twitter_favourites_h43,twitter_favourites_h44,twitter_favourites_h45,twitter_favourites_h46,twitter_favourites_h47,twitter_favourites_h48
47119,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
34223,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
33727,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
33231,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
32735,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34227,6238,6238,6238,6238,6238,6238,6238,6238,6238,6238,6238,6238,6238,6238,6238,6238,6238,6238,6238,6238,6238,6238,6238,6238,6238,6238,6238,6238,6238,6238,6238,6238,6238,6238,6238,6251,6251,6259,6259,6259,6259,6259,6259,6259,6259,6265,6265,6265,6265
64979,3280,3280,3280,3280,3280,3279,3279,3279,3279,3279,3279,3279,3279,3279,3278,3278,3278,3274,3274,3274,3274,3274,3274,3274,3274,3274,3274,3274,3274,3274,3274,3274,3274,3274,3274,3274,3274,3274,3274,3274,3274,3273,3273,3271,3271,3271,3268,3268,3265
74403,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463,463
56547,2477,2477,2477,2477,2477,2477,2477,2477,2477,2477,2474,2474,2474,2474,2474,2474,2474,2470,2470,2470,2458,2458,2458,2458,2458,2458,2458,2458,2458,2458,2458,2458,2458,2458,2458,2458,2458,2458,2458,2458,2458,2458,2458,2458,2458,2458,2458,2458,2458


*** Now we have the time series data up to 1 month in the past!
*** So now we can start preparing the data and sort the categories together?

In [50]:
print(x_final.shape)
x_final[-5:]

(80940, 1871)


,channelLink,channelTitle,currency,exchange,signalTime,pumptime_edited,timestamp,timestamp_h0,timestamp_h1,timestamp_h2,timestamp_h3,timestamp_h4,timestamp_h5,timestamp_h6,timestamp_h7,timestamp_h8,timestamp_h9,timestamp_h10,timestamp_h11,timestamp_h12,timestamp_h13,timestamp_h14,timestamp_h15,timestamp_h16,timestamp_h17,timestamp_h18,timestamp_h19,timestamp_h20,timestamp_h21,timestamp_h22,timestamp_h23,timestamp_h24,timestamp_h25,timestamp_h26,timestamp_h27,timestamp_h28,timestamp_h29,timestamp_h30,timestamp_h31,timestamp_h32,timestamp_h33,timestamp_h34,timestamp_h35,timestamp_h36,timestamp_h37,timestamp_h38,timestamp_h39,timestamp_h40,timestamp_h41,timestamp_h42,timestamp_h43,timestamp_h44,timestamp_h45,timestamp_h46,timestamp_h47,timestamp_h48,comments_h0,posts_h0,followers_h0,points_h0,overview_page_views_h0,analysis_page_views_h0,markets_page_views_h0,charts_page_views_h0,trades_page_views_h0,forum_page_views_h0,influence_page_views_h0,total_page_views_h0,fb_likes_h0,fb_talking_about_h0,twitter_followers_h0,twitter_following_h0,twitter_lists_h0,twitter_favourites_h0,twitter_statuses_h0,reddit_subscribers_h0,reddit_active_users_h0,reddit_posts_per_hour_h0,reddit_posts_per_day_h0,reddit_comments_per_hour_h0,reddit_comments_per_day_h0,code_repo_stars_h0,code_repo_forks_h0,code_repo_subscribers_h0,code_repo_open_pull_issues_h0,code_repo_closed_pull_issues_h0,code_repo_open_issues_h0,code_repo_closed_issues_h0,code_repo_contributors_h0,date_h0,date_utc_h0,coin_name_h0,coin_id_h0,comments_h1,posts_h1,followers_h1,points_h1,overview_page_views_h1,analysis_page_views_h1,markets_page_views_h1,charts_page_views_h1,trades_page_views_h1,forum_page_views_h1,influence_page_views_h1,total_page_views_h1,fb_likes_h1,fb_talking_about_h1,twitter_followers_h1,twitter_following_h1,twitter_lists_h1,twitter_favourites_h1,twitter_statuses_h1,reddit_subscribers_h1,reddit_active_users_h1,reddit_posts_per_hour_h1,reddit_posts_per_day_h1,reddit_comments_per_hour_h1,reddit_comments_per_day_h1,code_repo_stars_h1,code_repo_forks_h1,code_repo_subscribers_h1,code_repo_open_pull_issues_h1,code_repo_closed_pull_issues_h1,code_repo_open_issues_h1,code_repo_closed_issues_h1,code_repo_contributors_h1,date_h1,date_utc_h1,coin_name_h1,coin_id_h1,comments_h2,posts_h2,followers_h2,points_h2,overview_page_views_h2,analysis_page_views_h2,markets_page_views_h2,charts_page_views_h2,trades_page_views_h2,forum_page_views_h2,influence_page_views_h2,total_page_views_h2,fb_likes_h2,fb_talking_about_h2,twitter_followers_h2,twitter_following_h2,twitter_lists_h2,twitter_favourites_h2,twitter_statuses_h2,reddit_subscribers_h2,reddit_active_users_h2,reddit_posts_per_hour_h2,reddit_posts_per_day_h2,reddit_comments_per_hour_h2,reddit_comments_per_day_h2,code_repo_stars_h2,code_repo_forks_h2,code_repo_subscribers_h2,code_repo_open_pull_issues_h2,code_repo_closed_pull_issues_h2,code_repo_open_issues_h2,code_repo_closed_issues_h2,code_repo_contributors_h2,date_h2,date_utc_h2,coin_name_h2,coin_id_h2,comments_h3,posts_h3,followers_h3,points_h3,overview_page_views_h3,analysis_page_views_h3,markets_page_views_h3,charts_page_views_h3,trades_page_views_h3,forum_page_views_h3,influence_page_views_h3,total_page_views_h3,fb_likes_h3,fb_talking_about_h3,twitter_followers_h3,twitter_following_h3,twitter_lists_h3,twitter_favourites_h3,twitter_statuses_h3,reddit_subscribers_h3,reddit_active_users_h3,reddit_posts_per_hour_h3,reddit_posts_per_day_h3,reddit_comments_per_hour_h3,reddit_comments_per_day_h3,code_repo_stars_h3,code_repo_forks_h3,code_repo_subscribers_h3,code_repo_open_pull_issues_h3,code_repo_closed_pull_issues_h3,code_repo_open_issues_h3,code_repo_closed_issues_h3,code_repo_contributors_h3,date_h3,date_utc_h3,coin_name_h3,coin_id_h3,comments_h4,posts_h4,followers_h4,points_h4,overview_page_views_h4,analysis_page_views_h4,markets_page_views_h4,charts_page_views_h4,trades_page_views_h4,forum_page_views_h4,influence_page_views_h4,total_page_views_h4,fb_likes_h4,fb_talking_about_h4,twitt

Addiitional steps to process data:
* Group the data by types of social media

In [51]:
code = ['code_repo_stars',
'code_repo_forks' ,
'code_repo_subscribers',
'code_repo_contributors',
'code_repo_open_pull_issues',
'code_repo_closed_pull_issues',
'code_repo_open_issues',
'code_repo_closed_issues']

twitter = ['twitter_followers',
'twitter_following',
'twitter_lists',
'twitter_favourites',
'twitter_statuses']

reddit = ['reddit_subscribers',
'reddit_active_users',
'reddit_posts_per_hour',
'reddit_posts_per_day',
'reddit_comments_per_hour',
'reddit_comments_per_day'
]

fb = ['fb_likes','fb_talking_about']

market = [
'comments',
'posts',
'followers' ,
'points',
'overview_page_views',
'analysis_page_views',
'markets_page_views',
'charts_page_views',
'trades_page_views',
'forum_page_views',
'influence_page_views',
'total_page_views'
]


In [52]:
h = 49
col_dict = dict()
for f in ['code','twitter','reddit','fb','market']:
    dt = dict()
    for i in eval(f):
        dt[i] = [i+"_h"+str(y) for y in range(h)]
    col_dict[f] = dt

In [53]:
core = ['channelLink','channelTitle','currency','exchange',
        'timestamp','signalTime','pumptime_edited','coin_name', 'coin_id']

In [54]:
col_dict['core'] = core

In [55]:
col_dict[list(col_dict.keys())[0]].keys()

dict_keys(['code_repo_stars', 'code_repo_forks', 'code_repo_subscribers', 'code_repo_contributors', 'code_repo_open_pull_issues', 'code_repo_closed_pull_issues', 'code_repo_open_issues', 'code_repo_closed_issues'])

In [56]:
code_d = [x+"_h"+str(y) for x in code for y in range(h)]
twitter_d = [x+"_h"+str(y) for x in twitter for y in range(h)]
reddit_d = [x+"_h"+str(y) for x in reddit for y in range(h)]
fb_d = [x+"_h"+str(y) for x in fb for y in range(h)]
market_d = [x+"_h"+str(y) for x in market for y in range(h)]
timestamp = ['timestamp'+'_h'+str(y)  for y in range(h)]

In [57]:
#Omit timestamp for now
edited_cols = np.array(core + code_d + twitter_d + reddit_d + fb_d + market_d)
#MUST MAKE SURE THE ELEMENTS ARE IN THE SAME ORDER AS IN COL_DICT ABOVE!!!!
social_features = np.array(code_d + twitter_d + reddit_d + fb_d + market_d)
print(len(edited_cols))
unique_cols = np.unique(edited_cols)
print(len(unique_cols))

1626
1626


## Make sure all the importmnat columns are preserved

In [58]:
orig_cols = x_final.columns
for i in orig_cols:
    if i not in unique_cols:
        print(i)

timestamp_h0
timestamp_h1
timestamp_h2
timestamp_h3
timestamp_h4
timestamp_h5
timestamp_h6
timestamp_h7
timestamp_h8
timestamp_h9
timestamp_h10
timestamp_h11
timestamp_h12
timestamp_h13
timestamp_h14
timestamp_h15
timestamp_h16
timestamp_h17
timestamp_h18
timestamp_h19
timestamp_h20
timestamp_h21
timestamp_h22
timestamp_h23
timestamp_h24
timestamp_h25
timestamp_h26
timestamp_h27
timestamp_h28
timestamp_h29
timestamp_h30
timestamp_h31
timestamp_h32
timestamp_h33
timestamp_h34
timestamp_h35
timestamp_h36
timestamp_h37
timestamp_h38
timestamp_h39
timestamp_h40
timestamp_h41
timestamp_h42
timestamp_h43
timestamp_h44
timestamp_h45
timestamp_h46
timestamp_h47
timestamp_h48
date_h0
date_utc_h0
coin_name_h0
coin_id_h0
date_h1
date_utc_h1
coin_name_h1
coin_id_h1
date_h2
date_utc_h2
coin_name_h2
coin_id_h2
date_h3
date_utc_h3
coin_name_h3
coin_id_h3
date_h4
date_utc_h4
coin_name_h4
coin_id_h4
date_h5
date_utc_h5
coin_name_h5
coin_id_h5
date_h6
date_utc_h6
coin_name_h6
coin_id_h6
date_h7
date_utc

In [59]:
x_final = x_final[edited_cols]
x_final = x_final.reset_index(drop=True)

In [60]:
x_final.to_pickle("social_data_training_hour.pkl")

In [61]:
x_final.head()

,channelLink,channelTitle,currency,exchange,timestamp,signalTime,pumptime_edited,coin_name,coin_id,code_repo_stars_h0,code_repo_stars_h1,code_repo_stars_h2,code_repo_stars_h3,code_repo_stars_h4,code_repo_stars_h5,code_repo_stars_h6,code_repo_stars_h7,code_repo_stars_h8,code_repo_stars_h9,code_repo_stars_h10,code_repo_stars_h11,code_repo_stars_h12,code_repo_stars_h13,code_repo_stars_h14,code_repo_stars_h15,code_repo_stars_h16,code_repo_stars_h17,code_repo_stars_h18,code_repo_stars_h19,code_repo_stars_h20,code_repo_stars_h21,code_repo_stars_h22,code_repo_stars_h23,code_repo_stars_h24,code_repo_stars_h25,code_repo_stars_h26,code_repo_stars_h27,code_repo_stars_h28,code_repo_stars_h29,code_repo_stars_h30,code_repo_stars_h31,code_repo_stars_h32,code_repo_stars_h33,code_repo_stars_h34,code_repo_stars_h35,code_repo_stars_h36,code_repo_stars_h37,code_repo_stars_h38,code_repo_stars_h39,code_repo_stars_h40,code_repo_stars_h41,code_repo_stars_h42,code_repo_stars_h43,code_repo_stars_h44,code_repo_stars_h45,code_repo_stars_h46,code_repo_stars_h47,code_repo_stars_h48,code_repo_forks_h0,code_repo_forks_h1,code_repo_forks_h2,code_repo_forks_h3,code_repo_forks_h4,code_repo_forks_h5,code_repo_forks_h6,code_repo_forks_h7,code_repo_forks_h8,code_repo_forks_h9,code_repo_forks_h10,code_repo_forks_h11,code_repo_forks_h12,code_repo_forks_h13,code_repo_forks_h14,code_repo_forks_h15,code_repo_forks_h16,code_repo_forks_h17,code_repo_forks_h18,code_repo_forks_h19,code_repo_forks_h20,code_repo_forks_h21,code_repo_forks_h22,code_repo_forks_h23,code_repo_forks_h24,code_repo_forks_h25,code_repo_forks_h26,code_repo_forks_h27,code_repo_forks_h28,code_repo_forks_h29,code_repo_forks_h30,code_repo_forks_h31,code_repo_forks_h32,code_repo_forks_h33,code_repo_forks_h34,code_repo_forks_h35,code_repo_forks_h36,code_repo_forks_h37,code_repo_forks_h38,code_repo_forks_h39,code_repo_forks_h40,code_repo_forks_h41,code_repo_forks_h42,code_repo_forks_h43,code_repo_forks_h44,code_repo_forks_h45,code_repo_forks_h46,code_repo_forks_h47,code_repo_forks_h48,code_repo_subscribers_h0,code_repo_subscribers_h1,code_repo_subscribers_h2,code_repo_subscribers_h3,code_repo_subscribers_h4,code_repo_subscribers_h5,code_repo_subscribers_h6,code_repo_subscribers_h7,code_repo_subscribers_h8,code_repo_subscribers_h9,code_repo_subscribers_h10,code_repo_subscribers_h11,code_repo_subscribers_h12,code_repo_subscribers_h13,code_repo_subscribers_h14,code_repo_subscribers_h15,code_repo_subscribers_h16,code_repo_subscribers_h17,code_repo_subscribers_h18,code_repo_subscribers_h19,code_repo_subscribers_h20,code_repo_subscribers_h21,code_repo_subscribers_h22,code_repo_subscribers_h23,code_repo_subscribers_h24,code_repo_subscribers_h25,code_repo_subscribers_h26,code_repo_subscribers_h27,code_repo_subscribers_h28,code_repo_subscribers_h29,code_repo_subscribers_h30,code_repo_subscribers_h31,code_repo_subscribers_h32,code_repo_subscribers_h33,code_repo_subscribers_h34,code_repo_subscribers_h35,code_repo_subscribers_h36,code_repo_subscribers_h37,code_repo_subscribers_h38,code_repo_subscribers_h39,code_repo_subscribers_h40,code_repo_subscribers_h41,code_repo_subscribers_h42,code_repo_subscribers_h43,code_repo_subscribers_h44,code_repo_subscribers_h45,code_repo_subscribers_h46,code_repo_subscribers_h47,code_repo_subscribers_h48,code_repo_contributors_h0,code_repo_contributors_h1,code_repo_contributors_h2,code_repo_contributors_h3,code_repo_contributors_h4,code_repo_contributors_h5,code_repo_contributors_h6,code_repo_contributors_h7,code_repo_contributors_h8,code_repo_contributors_h9,code_repo_contributors_h10,code_repo_contributors_h11,code_repo_contributors_h12,code_repo_contributors_h13,code_repo_contributors_h14,code_repo_contributors_h15,code_repo_contributors_h16,code_repo_contributors_h17,code_repo_contributors_h18,code_repo_contributors_h19,code_repo_contributors_h20,code_repo_contributors_h21,code_repo_contributors_h22,code_repo_contributors_h23,code_repo_contributors_h24,code_repo_contributors_h25,code_

## Save the info

In [62]:
f = open("col_dict_h.pkl","wb")
pickle.dump(col_dict,f)
f.close()

g = open("social_features_h.pkl","wb")
pickle.dump(social_features,g)
g.close()